In [6]:
import h5py

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import ROOT
from ROOT import gStyle

%jsroot on

In [7]:
with h5py.File("./msd_summary_file.h5", "r") as f:
    data = np.array(f["analysis"]["songs"])["duration"]
    data = data / 60

In [51]:
f = h5py.File("./msd_summary_file.h5", "r")

data = pd.DataFrame(np.array(f["analysis"]["songs"]))

data["duration"] = data["duration"] / 60

data[["duration"]].to_csv("./duration.csv")

In [8]:
canvas = ROOT.TCanvas("canvas")
num_count = len(data)
h = ROOT.TH1F("myHist", "Song duration", num_count, 0, 14)

for i in data:
    h.Fill(i)

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Warning in <TROOT::Append>: Replacing existing TH1: myHist (Potential memory leak).


In [9]:
f1 = ROOT.TF1("f1", "gaus+gaus(3)", 2, 6);
f1 = ROOT.TF1("f1", "gaus", 2, 6);
f1.SetParLimits(0, 160, 210)
f1.SetParLimits(1, 3.4, 3.7)
f1.SetParLimits(2, 0.5, 1.4)

# f1.SetParLimits(3, 50, 75)
# f1.SetParLimits(4, 3.5, 5.4)
# f1.SetParLimits(5, 2.0, 3.2)
# h.fit("gaus")
h.Fit("f1", "WR", "", 2.5, 5)

chi2 = f1.GetChisquare()
print(chi2/num_count)
gStyle.SetOptFit(1111);
# canvas.SetOptFit(1011)

canvas.Draw()
h.Draw("h")

6.852308618279321

****************************************
Minimizer is Minuit / Migrad
Chi2                      =  6.85231e+06
NDf                       =         5849
Edm                       =  3.45043e-06
NCalls                    =           99
Constant                  =          160   +/-   0.000381432  	 (limited)
Mean                      =      3.61467   +/-   0.00575193   	 (limited)
Sigma                     =     0.859798   +/-   0.00540509   	 (limited)


Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 


In [10]:
# def fitfunction_poisson(x, par):
#     if par[2] != 0:
#         if ROOT.TMath.Gamma((x[0]/par[2])+1) != 0:
#             poisson = par[0] * TMath.Power(par[1], x) * (TMath.Exp(-par[1]))/TMath.Factorial(x) + [5] * TMath.Gaus(x,[3],[4])
#         else:
#             poisson = 0
#     else:
#         poisson = 0

#     return poisson


def fitfunction_poisson(x, par):
    return par[3] * TMath.Power(par[4], x) * (TMath.Exp(-par[4]))/TMath.Factorial(x) + par[2] * TMath.Gaus(x, par[0], par[1])

canvas = ROOT.TCanvas("canvas")
num_count = len(data)
h = ROOT.TH1F("myHist", "Song duration", num_count, 0, 14)

for i in data:
    h.Fill(i)

f1 = ROOT.TF1("f1", fitfunction_poisson, 1, 14);
f1.SetParLimits(3, 50, 200)
f1.SetParLimits(4, 1, 4.5)

f1.SetParLimits(0, 120, 200)
f1.SetParLimits(1, 3.4, 3.7)
f1.SetParLimits(2, 0.5, 1.4)

# h.fit("gaus")
h.Fit("f1", "WR", "", 0, 14)

chi2 = f1.GetChisquare()
print(chi2/num_count)
gStyle.SetOptFit(1111);
# canvas.SetOptFit(1011)

canvas.Draw()
h.Draw("h")

0.0


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Warning in <TROOT::Append>: Replacing existing TH1: myHist (Potential memory leak).
Error in <Fit>: function f1 has illegal number of parameters = 0


In [11]:
f2 = ROOT.TF1("f2", "gaus(0)+gaus(3)", 2, 6);
f2.FixParameter(0, 200)
f2.FixParameter(1, 3.6)
f2.FixParameter(2, 1)

f2.FixParameter(3, 50)
f2.FixParameter(4, 4.2)
f2.FixParameter(5, 3)
f2.SetLineColor(6)
f2.SetRange(0, 14)
canvas.Draw()
f2.Draw()

In [ ]:
[0]*TMath::Power(([1]/[2]),(x/[2]))*(TMath::Exp( ([1]/[2])))/TMath::Gamma((x/[2])+1)

In [12]:
from ROOT import TMath

In [46]:
# def fitfunction_poisson(x, par):
#     if par[2] != 0:
#         if ROOT.TMath.Gamma((x[0]/par[2])+1) != 0:
#             poisson = par[0] * ROOT.TMath.Power((par[1]/par[2]),(x[0]/par[2])) * (ROOT.TMath.Exp(-(par[1]/par[2])))/ROOT.TMath.Gamma((x[0]/par[2])+1)
#         else:
#             poisson = 0
#     else:
#         poisson = 0

#     return poisson

def fitfunction_poisson(x, par):
#     return par[3] * TMath.Power(par[4], x) * (TMath.Exp(-par[4]))/TMath.Factorial(x) + par[2] * TMath.Gaus(x, par[0], par[1])
    return par[2] * TMath.Gaus(x, par[0], par[1])

PoissonFit = ROOT.TF1('PoissonFit', fitfunction_poisson, 0, 14, 3)
PoissonFit.SetParameter(0, 20)
PoissonFit.SetParameter(1, 1)
PoissonFit.SetParameter(2, 0.3)
# PoissonFit.SetParameter(3, 1)
# PoissonFit.SetParameter(4, 4)

# PoissonFit.SetParameter(2, 2)
# PoissonFit.SetParNames('par0', 'par1', 'par2')
# PoissonFit.SetLineColor(4)
# PoissonFit.SetLineWidth(2)
# PoissonFit.SetLineStyle(2)

canvas.Draw()
PoissonFit.Draw()

# gStyle.SetOptFit("0111")
# ROOT.gPad.Modified()

# values.Fit('PoissonFit', 'R')
# PoissonFit.Draw('same')

In [44]:
def fitfunction_poisson(x, par):
#     return par[3] * TMath.Power(par[4], x) * (TMath.Exp(-par[4]))/TMath.Factorial(x) + par[2] * TMath.Gaus(x, par[0], par[1])
#     return par[2] * TMath.Gaus(x, par[0], par[1])
#     return par[0]*TMath.Exp(-0.5*TMath.Power((x-par[1])/par[2],2))
    return "[0]*TMath::Exp(-0.5*TMath::Power((x-[1])/[2],2))"

canvas = ROOT.TCanvas("canvas")

# PoissonFit = ROOT.TF1('PoissonFit', "[0]*exp(-0.5*pow((x-[1])/[2],2)) + [3]*exp(-[4]*x)", 0, 14, 3)
# PoissonFit = ROOT.TF1('PoissonFit', "[0]*TMath::Exp(-0.5*TMath::Power((x-[1])/[2],2))", 0, 14, 3)

# PoissonFit = ROOT.TF1('PoissonFit', "[0]*exp(-0.5*pow((x-[1])/[2],2))+gaus", 0, 14, 3)
# PoissonFit = ROOT.TF1('PoissonFit', "[3]*pow([4], x)*(exp(-[4]))/gamma(x) + gaus(0)", 0, 14, 5)
PoissonFit = ROOT.TF1('PoissonFit', "poisson + gaus(0)", 0, 14, 5)
PoissonFit.FixParameter(0, 20)
PoissonFit.FixParameter(1, 3)
PoissonFit.FixParameter(2, 3)
PoissonFit.FixParameter(3, 1)
PoissonFit.FixParameter(4, 4)

PoissonFit.SetLineColor(6)
PoissonFit.SetRange(0, 14)

canvas.Draw()
PoissonFit.Draw()


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
input_line_176:2:79: error: use of undeclared identifier 'poisson'
Double_t TFormula____id15781373761806914400(Double_t *x,Double_t *p){ return {poisson}+p[0]*TMath::Exp(-0.5*((x[0]-p[1])/p[2])*((x[0]-p[1])/p[2])) ; }
                                                                              ^
input_line_177:2:79: error: use of undeclared identifier 'poisson'
Double_t TFormula____id15781373761806914400(Double_t *x,Double_t *p){ return {poisson}+p[0]*TMath::Exp(-0.5*((x[0]-p[1])/p[2])*((x[0]-p[1])/p[2])) ; }
                                                                              ^
Error in <prepareMethod>: Can't compile function TFormula____id15781373761806914400 prototype with arguments Double_t*,Double_t*
Error in <TFormula::InputFormulaIntoCling>: Error compiling formula expression in Cling
Error in <TFormula::ProcessFormula>: "poisson" has not been matched in the formula expression
Error in <TFormula: